In [6]:
# This is for my own setup

# !ls -la /content
# !git clone https://github.com/mehrdad-mirpourian/nlu_hw2.git
# !ls -la /content/nlu_hw2


# import os
# os.chdir("/content/nlu_hw2")
# print("Current directory:", os.getcwd())  # Should print /content/nlu_hw2

# !ls -la

total 156
drwxr-xr-x 3 root root   4096 Mar  8 18:40 .
drwxr-xr-x 1 root root   4096 Mar  8 18:40 ..
drwxr-xr-x 8 root root   4096 Mar  8 18:40 .git
-rw-r--r-- 1 root root 124125 Mar  8 18:40 hw2-pset.ipynb
-rw-r--r-- 1 root root    589 Mar  8 18:40 My_Checks.ipynb
-rw-r--r-- 1 root root   4903 Mar  8 18:40 README.md
-rw-r--r-- 1 root root   1310 Mar  8 18:40 test_model.py
-rw-r--r-- 1 root root   3952 Mar  8 18:40 train_model.py


In [7]:
! pip install datasets evaluate optuna --quiet # install datasets if it is not included in your environment

In [8]:
import torch
from collections.abc import Iterable
from datasets import load_dataset

# Model and tokenizer from 🤗 Transformers
from transformers import AutoModelForSequenceClassification, \
    BertForSequenceClassification, BertTokenizerFast


In [ ]:
# Mehi: When I start running the model I need to check this part and run it
# Code you will write for this assignment
from train_model import init_model, preprocess_dataset, init_trainer
from test_model import init_tester

In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = BertForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# This code does exactly the same thing as the previous code cell
model = AutoModelForSequenceClassification.from_pretrained(
    "prajjwal1/bert-tiny", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
tokenizer = BertTokenizerFast.from_pretrained("prajjwal1/bert-tiny")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
# Because 🤗 Transformers supports multiple deep learning libraries, you will
# need to use the keyword parameter return_tensors in order to indicate that
# you want your inputs to be returned in PyTorch format.
inputs = tokenizer(["Hello world!", "How are you?"], padding=True,
                   return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102,    0],
        [ 101, 2129, 2024, 2017, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]])}

In [14]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

print(outputs, end="\n\n")

# Use the dot operator to access parts of the output
print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3228, -0.0692],
        [-0.1899, -0.0947]]), hidden_states=None, attentions=None)

tensor([[-0.3228, -0.0692],
        [-0.1899, -0.0947]])
